In [1]:
import wrangle as wgl
import split_scale as ss
import evaluate as ev
import explore as exp
from env import get_db_url
import pandas as pd
import numpy as np
import seaborn as sns
import feature_engineering as fe
from statsmodels.formula.api import ols

In [2]:
query = '''
select *
from properties_2017
join predictions_2017 as pred
using(parcelid)
where pred.`transactiondate` like '%%-05-%%' or pred.`transactiondate` like '%%-06-%%'
'''
df = pd.read_sql(query, get_db_url('zillow'))

#drop object columns
df = df.select_dtypes(exclude=['object'])

In [3]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21944 entries, 0 to 21943
Data columns (total 58 columns):
parcelid                        21944 non-null int64
id                              21944 non-null int64
airconditioningtypeid           7088 non-null float64
architecturalstyletypeid        49 non-null float64
basementsqft                    12 non-null float64
bathroomcnt                     21938 non-null float64
bedroomcnt                      21938 non-null float64
buildingclasstypeid             4 non-null float64
buildingqualitytypeid           13949 non-null float64
calculatedbathnbr               21762 non-null float64
decktypeid                      179 non-null float64
finishedfloor1squarefeet        1765 non-null float64
calculatedfinishedsquarefeet    21883 non-null float64
finishedsquarefeet12            20951 non-null float64
finishedsquarefeet13            12 non-null float64
finishedsquarefeet15            810 non-null float64
finishedsquarefeet50            17

,parcelid,id,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyyear,censustractandblock,id,logerror
count,2.194400e+04,2.194400e+04,7088.000000,49.000000,12.000000,21938.000000,21938.000000,4.00,13949.000000,21762.000000,...,43.0,2.190200e+04,2.193800e+04,21938.0,2.193800e+04,21937.000000,741.000000,2.185700e+04,21944.000000,21944.000000
mean,1.301764e+07,1.509334e+06,1.852709,7.346939,756.333333,2.319924,3.065685,3.75,6.558105,2.338089,...,1.0,1.933349e+05,5.034094e+05,2016.0,3.103918e+05,6153.475476,14.062078,6.049357e+13,42481.528664,0.012441
std,3.404377e+06,8.598458e+05,3.025647,3.112816,424.486927,0.995531,1.125493,0.50,1.718882,0.978087,...,0.0,2.357308e+05,6.450342e+05,0.0,4.644094e+05,7484.387289,1.489215,2.077894e+11,6478.443523,0.168442
min,1.071210e+07,3.490000e+02,1.000000,2.000000,280.000000,0.000000,0.000000,3.00,1.000000,1.000000,...,1.0,1.290000e+02,1.050400e+04,2016.0,2.250000e+03,19.920000,8.000000,6.037101e+13,117.000000,-4.453964
25%,1.153894e+07,7.722192e+05,1.000000,7.000000,548.500000,2.000000,2.000000,3.75,6.000000,2.000000,...,1.0,8.612650e+04,2.128388e+05,2016.0,8.930150e+04,2772.820000,14.000000,6.037312e+13,37024.750000,-0.026602
50%,1.253618e+07,1.524332e+06,1.000000,7.000000,670.000000,2.000000,3.000000,4.00,6.000000,2.000000,...,1.0,1.404325e+05,3.682350e+05,2016.0,2.092070e+05,4565.180000,15.000000,6.037621e+13,42517.500000,0.004400
75%,1.422253e+07,2.251325e+06,1.000000,7.000000,842.250000,3.000000,4.000000,4.00,8.000000,3.000000,...,1.0,2.250000e+05,5.888448e+05,2016.0,3.793890e+05,7169.480000,15.000000,6.059042e+13,48012.250000,0.036083
max,1.676878e+08,2.985043e+06,13.000000,21.000000,1809.000000,11.000000,12.000000,4.00,12.000000,11.000000,...,1.0,7.893568e+06,2.385837e+07,2016.0,1.635060e+07,276797.830000,15.000000,6.111009e+13,77613.000000,3.175688


In [4]:
#drop id columns
id_columns = []
for i in df.columns.tolist():
    if 'id' in i:
        id_columns += [i]
    
clean_df = df.drop(id_columns, axis = 1)
id_columns

['parcelid',
 'id',
 'airconditioningtypeid',
 'architecturalstyletypeid',
 'buildingclasstypeid',
 'buildingqualitytypeid',
 'decktypeid',
 'heatingorsystemtypeid',
 'pooltypeid10',
 'pooltypeid2',
 'pooltypeid7',
 'propertylandusetypeid',
 'regionidcity',
 'regionidcounty',
 'regionidneighborhood',
 'regionidzip',
 'storytypeid',
 'typeconstructiontypeid',
 'id']

In [5]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21944 entries, 0 to 21943
Data columns (total 39 columns):
basementsqft                    12 non-null float64
bathroomcnt                     21938 non-null float64
bedroomcnt                      21938 non-null float64
calculatedbathnbr               21762 non-null float64
finishedfloor1squarefeet        1765 non-null float64
calculatedfinishedsquarefeet    21883 non-null float64
finishedsquarefeet12            20951 non-null float64
finishedsquarefeet13            12 non-null float64
finishedsquarefeet15            810 non-null float64
finishedsquarefeet50            1765 non-null float64
finishedsquarefeet6             110 non-null float64
fips                            21938 non-null float64
fireplacecnt                    2432 non-null float64
fullbathcnt                     21762 non-null float64
garagecarcnt                    7368 non-null float64
garagetotalsqft                 7368 non-null float64
hashottuborspa            

In [6]:
clean_df.describe()

,basementsqft,bathroomcnt,bedroomcnt,calculatedbathnbr,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyyear,censustractandblock,logerror
count,12.000000,21938.000000,21938.000000,21762.000000,1765.000000,21883.000000,20951.000000,12.000000,810.00000,1765.000000,...,5046.000000,43.0,2.190200e+04,2.193800e+04,21938.0,2.193800e+04,21937.000000,741.000000,2.185700e+04,21944.000000
mean,756.333333,2.319924,3.065685,2.338089,1367.755241,1808.392268,1784.021908,1374.333333,2399.65679,1375.552408,...,1.437574,1.0,1.933349e+05,5.034094e+05,2016.0,3.103918e+05,6153.475476,14.062078,6.049357e+13,0.012441
std,424.486927,0.995531,1.125493,0.978087,639.562706,973.433723,936.842699,131.292410,1517.17168,659.394700,...,0.539029,0.0,2.357308e+05,6.450342e+05,0.0,4.644094e+05,7484.387289,1.489215,2.077894e+11,0.168442
min,280.000000,0.000000,0.000000,1.000000,58.000000,242.000000,242.000000,1056.000000,600.00000,63.000000,...,1.000000,1.0,1.290000e+02,1.050400e+04,2016.0,2.250000e+03,19.920000,8.000000,6.037101e+13,-4.453964
25%,548.500000,2.000000,2.000000,2.000000,968.000000,1194.000000,1182.000000,1333.000000,1630.50000,968.000000,...,1.000000,1.0,8.612650e+04,2.128388e+05,2016.0,8.930150e+04,2772.820000,14.000000,6.037312e+13,-0.026602
50%,670.000000,2.000000,3.000000,2.000000,1263.000000,1562.000000,1546.000000,1416.000000,2144.00000,1264.000000,...,1.000000,1.0,1.404325e+05,3.682350e+05,2016.0,2.092070e+05,4565.180000,15.000000,6.037621e+13,0.004400
75%,842.250000,3.000000,4.000000,3.000000,1612.000000,2145.000000,2110.000000,1440.000000,2839.50000,1613.000000,...,2.000000,1.0,2.250000e+05,5.888448e+05,2016.0,3.793890e+05,7169.480000,15.000000,6.059042e+13,0.036083
max,1809.000000,11.000000,12.000000,11.000000,6553.000000,35640.000000,15450.000000,1560.000000,35640.00000,6553.000000,...,3.000000,1.0,7.893568e+06,2.385837e+07,2016.0,1.635060e+07,276797.830000,15.000000,6.111009e+13,3.175688


In [7]:
#clean squarefeet columns
sqr_ft_columns = []
for i in clean_df.columns.tolist():
    if 'finishedsquarefeet' in i:
        sqr_ft_columns += [i]
        
clean_df['square_feet'] = clean_df[sqr_ft_columns].mean(axis = 1)
clean_df = clean_df.drop(sqr_ft_columns, axis = 1)

In [8]:
#drop columns with few values
suffecient_columns = clean_df.count(axis = 0)[clean_df.count(axis = 0) > 0].index.tolist()
clean_df = clean_df[suffecient_columns]
clean_df.describe()

,basementsqft,bathroomcnt,bedroomcnt,calculatedbathnbr,finishedfloor1squarefeet,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,...,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyyear,censustractandblock,logerror,square_feet
count,12.000000,21938.000000,21938.000000,21762.000000,1765.000000,21938.000000,2432.000000,21762.000000,7368.000000,7368.000000,...,43.0,2.190200e+04,2.193800e+04,21938.0,2.193800e+04,21937.000000,741.000000,2.185700e+04,21944.000000,21883.000000
mean,756.333333,2.319924,3.065685,2.338089,1367.755241,6049.062084,1.204359,2.269461,1.821797,355.498914,...,1.0,1.933349e+05,5.034094e+05,2016.0,3.103918e+05,6153.475476,14.062078,6.049357e+13,0.012441,1794.013176
std,424.486927,0.995531,1.125493,0.978087,639.562706,20.946966,0.490734,0.964464,0.593963,261.890880,...,0.0,2.357308e+05,6.450342e+05,0.0,4.644094e+05,7484.387289,1.489215,2.077894e+11,0.168442,964.500835
min,280.000000,0.000000,0.000000,1.000000,58.000000,6037.000000,1.000000,1.000000,0.000000,0.000000,...,1.0,1.290000e+02,1.050400e+04,2016.0,2.250000e+03,19.920000,8.000000,6.037101e+13,-4.453964,242.000000
25%,548.500000,2.000000,2.000000,2.000000,968.000000,6037.000000,1.000000,2.000000,2.000000,0.000000,...,1.0,8.612650e+04,2.128388e+05,2016.0,8.930150e+04,2772.820000,14.000000,6.037312e+13,-0.026602,1188.000000
50%,670.000000,2.000000,3.000000,2.000000,1263.000000,6037.000000,1.000000,2.000000,2.000000,437.000000,...,1.0,1.404325e+05,3.682350e+05,2016.0,2.092070e+05,4565.180000,15.000000,6.037621e+13,0.004400,1554.000000
75%,842.250000,3.000000,4.000000,3.000000,1612.000000,6059.000000,1.000000,3.000000,2.000000,498.000000,...,1.0,2.250000e+05,5.888448e+05,2016.0,3.793890e+05,7169.480000,15.000000,6.059042e+13,0.036083,2125.000000
max,1809.000000,11.000000,12.000000,11.000000,6553.000000,6111.000000,4.000000,11.000000,11.000000,3233.000000,...,1.0,7.893568e+06,2.385837e+07,2016.0,1.635060e+07,276797.830000,15.000000,6.111009e+13,3.175688,35640.000000


In [9]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21944 entries, 0 to 21943
Data columns (total 34 columns):
basementsqft                  12 non-null float64
bathroomcnt                   21938 non-null float64
bedroomcnt                    21938 non-null float64
calculatedbathnbr             21762 non-null float64
finishedfloor1squarefeet      1765 non-null float64
fips                          21938 non-null float64
fireplacecnt                  2432 non-null float64
fullbathcnt                   21762 non-null float64
garagecarcnt                  7368 non-null float64
garagetotalsqft               7368 non-null float64
hashottuborspa                480 non-null float64
latitude                      21938 non-null float64
longitude                     21938 non-null float64
lotsizesquarefeet             19642 non-null float64
poolcnt                       4658 non-null float64
poolsizesum                   269 non-null float64
rawcensustractandblock        21938 non-null float64
ro

In [10]:
#drop locations:
clean_df = clean_df.drop(['latitude', 'longitude', 'rawcensustractandblock', 'censustractandblock'], axis = 1)

In [11]:
#drop redundancies
clean_df = clean_df.drop(['calculatedbathnbr', 'finishedfloor1squarefeet', 'roomcnt', 'threequarterbathnbr', 'structuretaxvaluedollarcnt', 'taxvaluedollarcnt', 'landtaxvaluedollarcnt'], axis = 1)

In [12]:
clean_df.describe()

,basementsqft,bathroomcnt,bedroomcnt,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,lotsizesquarefeet,...,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,assessmentyear,taxamount,taxdelinquencyyear,logerror,square_feet
count,12.000000,21938.000000,21938.000000,21938.000000,2432.000000,21762.000000,7368.000000,7368.000000,480.0,1.964200e+04,...,724.000000,23.000000,21857.000000,5046.000000,43.0,21938.0,21937.000000,741.000000,21944.000000,21883.000000
mean,756.333333,2.319924,3.065685,6049.062084,1.204359,2.269461,1.821797,355.498914,1.0,2.788102e+04,...,308.580110,215.739130,1968.798554,1.437574,1.0,2016.0,6153.475476,14.062078,0.012441,1794.013176
std,424.486927,0.995531,1.125493,20.946966,0.490734,0.964464,0.593963,261.890880,0.0,9.215600e+04,...,222.247773,171.218419,23.873276,0.539029,0.0,0.0,7484.387289,1.489215,0.168442,964.500835
min,280.000000,0.000000,0.000000,6037.000000,1.000000,1.000000,0.000000,0.000000,1.0,4.350000e+02,...,11.000000,28.000000,1824.000000,1.000000,1.0,2016.0,19.920000,8.000000,-4.453964,242.000000
25%,548.500000,2.000000,2.000000,6037.000000,1.000000,2.000000,2.000000,0.000000,1.0,5.696000e+03,...,176.000000,69.000000,1953.000000,1.000000,1.0,2016.0,2772.820000,14.000000,-0.026602,1188.000000
50%,670.000000,2.000000,3.000000,6037.000000,1.000000,2.000000,2.000000,437.000000,1.0,7.204000e+03,...,250.000000,174.000000,1970.000000,1.000000,1.0,2016.0,4565.180000,15.000000,0.004400,1554.000000
75%,842.250000,3.000000,4.000000,6059.000000,1.000000,3.000000,2.000000,498.000000,1.0,1.160000e+04,...,364.000000,348.000000,1987.000000,2.000000,1.0,2016.0,7169.480000,15.000000,0.036083,2125.000000
max,1809.000000,11.000000,12.000000,6111.000000,4.000000,11.000000,11.000000,3233.000000,1.0,3.589145e+06,...,1799.000000,648.000000,2015.000000,3.000000,1.0,2016.0,276797.830000,15.000000,3.175688,35640.000000


In [13]:
#change nans in count columns to zeros
clean_df.fireplacecnt.fillna(0, inplace = True)
clean_df.garagecarcnt.fillna(0, inplace = True)
clean_df.garagecarcnt.fillna(0, inplace = True)
clean_df.garagetotalsqft.fillna(0, inplace = True)
clean_df.poolcnt.fillna(0, inplace = True)
clean_df.numberofstories.fillna(1, inplace = True)


In [14]:
clean_df.dropna().count()

basementsqft          0
bathroomcnt           0
bedroomcnt            0
fips                  0
fireplacecnt          0
fullbathcnt           0
garagecarcnt          0
garagetotalsqft       0
hashottuborspa        0
lotsizesquarefeet     0
poolcnt               0
poolsizesum           0
unitcnt               0
yardbuildingsqft17    0
yardbuildingsqft26    0
yearbuilt             0
numberofstories       0
fireplaceflag         0
assessmentyear        0
taxamount             0
taxdelinquencyyear    0
logerror              0
square_feet           0
dtype: int64

In [15]:
#need housing prices(target variable)
clean_df = clean_df.join(df.id, how = 'inner').dropna()
clean_df = clean_df.merge(df[['id', 'taxvaluedollarcnt']], on = 'id').dropna()

ValueError: The column label 'id' is not unique.

In [ ]:
features = clean_df.columns.tolist()[:-1]
target = [clean_df.columns.tolist()[-1]]
features.remove('taxamount')
features

In [ ]:
train, test = ss.split_my_data(clean_df)

In [ ]:
scaler, train[features], test[features] = ss.standard_scaler(train[features],test[features])

In [ ]:
fe.lasso_cv_coef(train[features], train[target])

In [ ]:
regr = ols('taxvaluedollarcnt ~ bathroomcnt + bedroomcnt + fips + fireplacecnt + fullbathcnt + garagecarcnt + garagetotalsqft + poolcnt+unitcnt + yearbuilt + numberofstories + assessmentyear + square_feet', data = train).fit()
train['yhat'] = regr.predict(train)

In [ ]:
ev.model_significance(train[target], train['yhat'], regr)

In [ ]:
regr.summary()